In [4]:
import pandas as pd
import numpy as np
dc_listings = pd.read_csv('Downloads/listings.csv')
stripped_commas = dc_listings['price'].str.replace(',', '')
stripped_dollars = stripped_commas.str.replace('$', '')
dc_listings['price'] = stripped_dollars.astype('float')
dc_listings = dc_listings.loc[np.random.permutation(len(dc_listings))]

def predict_price(new_listing):
    temp_df = dc_listings.copy()
    ## Complete the function.
    temp_df['distance']=temp_df['accommodates'].apply(lambda x:abs(x-new_listing))
    temp_df=temp_df.sort_values(by=['distance'])
    predict=temp_df['price'].iloc[0:5].mean()
    return predict

acc_one = predict_price(1)
acc_two = predict_price(2)
acc_four = predict_price(4)